In [1]:
import langchain
from langchain_together import Together
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

from langchain_community.vectorstores import FAISS
# from langchain_chroma import Chroma
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

c:\Users\HamzaJadoon\anaconda3\envs\myenv\Lib\site-packages\langchain_together\__init__.py:3: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_together.chat_models import ChatTogether
c:\Users\HamzaJadoon\anaconda3\envs\myenv\Lib\site-packages\pydantic\_internal\_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
  warnings.warn(message, UserWarning)


PydanticUserError: The `__modify_schema__` method is not supported in Pydantic v2. Use `__get_pydantic_json_schema__` instead in class `SecretStr`.

For further information visit https://errors.pydantic.dev/2.9/u/custom-json-schema

<!-- - ## Go to https://api.together.ai
- ## Make an account and sign in
- ## Go to settings/api_keys. Generate one and use it as together_api_key -->

- **Get a Together API Key:**
    * Visit the Together.ai API website: https://api.together.ai
    * Create an account and sign in.
    * Navigate to your settings and then the API keys section.
    * Generate a new API key and copy it for later use. 

**Note:** Replace together_api_key with your actual key in the code snippet.



In [12]:
llm = Together(
    model="META-LLAMA/LLAMA-2-7B-CHAT-HF",
    together_api_key="24cdbdf50106e08f6ba3328ac07f97a73eb440ae36da6cdd72f9b091ccca850a"
)

model = ChatOpenAI(
    base_url="https://api.together.xyz/v1",
    api_key="24cdbdf50106e08f6ba3328ac07f97a73eb440ae36da6cdd72f9b091ccca850a",
    model="META-LLAMA/LLAMA-3-8B-CHAT-HF",
)

c:\Users\HamzaJadoon\anaconda3\envs\myenv\Lib\site-packages\langchain_together\llms.py:93: UserWarning: The completions endpoint, has 'max_tokens' as required argument. The default value is being set to 200 Consider setting this value, when initializing LLM
  warnings.warn(


## **Now suppose you want to chat with the model by providing it with some additional context in the form of text**

#### Vector store takes in two inputs: Data (simple text in our case), and pre-trained embedding model (ollamaembeddings in our case). ```It then embeds the provided text```
#### First up create a vector store and store your text in it. To use this vector store, you must retrieve it and store in a variable. 

In [ ]:
# Creating vector store
vectorstore = DocArrayInMemorySearch.from_texts(
    ["alex is looking for a job", "alex likes to visit parks becuase they are green"],
    embedding = OllamaEmbeddings(),
)
retriever = vectorstore.as_retriever()

In [4]:
template = """Answer the question based only on the following context: {context}
Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
output_parser = StrOutputParser()

**You will have to understand how the following work:**

 * RunnableParallel
 * RunnablePassthrough

**It has been explained to you in the following image:**

<img src="img_.jpg" alt="Alt text for image" style="width: 800px; height: 600px;">

In [5]:
# RunnableParallel runs multiple tasks at the same time
# RunnablePassthrough simply passes on the retrieved text/doc into "context"

setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)
# Notice how chain is created where setup_and_retrieval is connected to prompts chain
chain = setup_and_retrieval | prompt | model | output_parser

chain.invoke("what is alex doing")

'According to the context, Alex is looking for a job.'

In [6]:
chain.invoke("why does alex like to visit parks")

'According to the context, Alex likes to visit parks because they are green.'

# Now Lets up our game and implement a simple RAG model

<span style="font-size: 18px;">RAG (Retrieval-Augmented Generation) is a technique where a system first looks up relevant information from provided sources (websites/pdf etc) and then uses that information to generate reponse</span>
#### Following creates two vector stores: 
 * website
 * pdf file.

In [3]:
# local pdf file
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("QA-with-RAG/basic.pdf")
pages = loader.load_and_split()

In [4]:
embeddings = OllamaEmbeddings()
text_splitter = RecursiveCharacterTextSplitter()
pdf_doc = text_splitter.split_documents(pages)

### Now you can use many different types of vector stores like chroma etc. I used FAISS

In [5]:
prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context: 
<context> 
{context} 
</context>

Question: {input}""")

<img src="work.jpg" alt="Alt text for image" style="width: 800px; height: 600px;">

In [8]:
pdf_vector = FAISS.from_documents(pdf_doc, embeddings)
pdf_retriever = pdf_vector.as_retriever()

from langchain.chains.combine_documents import create_stuff_documents_chain
pdf_document_chain = create_stuff_documents_chain(llm, prompt)

from langchain.chains import create_retrieval_chain
pdf_retrieval_chain = create_retrieval_chain(pdf_retriever, pdf_document_chain)

In [9]:
response = pdf_retrieval_chain.invoke({"input": "Who is alex?"})

In [10]:
response

{'input': 'Who is alex?',
 'context': [Document(page_content="Once upon a time in the bustling city of Brooksville, Alex, a dedicated software engineer, found \nhimself facing unexpected adversity. Alex had poured his heart and soul into his job at Tech \nInnovations, a startup renowned for its cutting -edge technology solutions. He was the go -to guy \nfor troubleshooting, innovation, and camaraderie in the office.  \nHowever, one gloomy Monday morning, Alex was called into the CEO's office. His heart raced \nwith anticipation, but his hopes were shattered when he was informed that due to restructuring, \nhis position was being eliminated. Shock washed over him like an icy wave crashing onto the \nshore. His mind buzzed with disbelief and fear as he absorbed the reality of his sudden \nunemployment.  \nFeeling lost and uncertain about the future, Alex wandered the streets of Brooksville, his mind \nclouded with worry. Days turned into weeks, and the weight of unemployment hung heavy o